In [1]:
import requests
import pandas as pd

import json

## Calcular tarifas 

In [2]:
tarifas_auto = pd.read_excel('./dados/comex/tabela_tarifas_automotivas_hts (1).xlsx', dtype=str)

In [3]:
tarifas_auto.shape

(51, 5)

In [4]:
tarifas_auto.head(2)

,Codigo_HTSUS,Categoria_Tarifaria,Codigo_Tarifa_Especial_Cap99,Aliquota,Condicao
0,8703.22.01,Tarifa Geral,9903.94.01,25% sobre valor total,Padrão para todos os veículos importados.
1,8703.22.01,Exceção USMCA,9903.94.03,25% sobre conteúdo não-americano,Requer elegibilidade sob o USMCA e aprovação d...


In [5]:
tarifas_auto['SH6'] = tarifas_auto['Codigo_HTSUS'].str.replace('.', '', regex=False).str[0:6]

tarifas_auto = tarifas_auto[['SH6', 'Aliquota']].drop_duplicates()

In [6]:
tarifas_auto = tarifas_auto[['SH6']].drop_duplicates()

In [7]:
tarifas_auto['Instrumento'] = 'Seção 232'
tarifas_auto['Tarifa Desc'] = "10% (base) para veículos antigos ou conteúdo americano a 25%"
tarifas_auto['Tarifa'] = 0.25
tarifas_auto['Vigência'] = 'Em análise'


In [8]:
ncm = pd.read_csv('./dados/comex/NCM (2).csv',sep=';',dtype={'CO_SH6': str, 'CO_ISIC_CLASSE': str})

ncm_sh6 = ncm[['CO_SH6']].drop_duplicates()
ncm_sh6['CO_SH6'] = ncm_sh6['CO_SH6'].str.strip().str.zfill(6)
ncm_sh6.rename(columns={'CO_SH6': 'SH6'}, inplace=True)

In [9]:


harmonized = pd.read_excel('./dados/comex/Harmonized Tariff Schedule of the United States (HTSUS) not covered by the action.xlsx')
harmonized['SH6'] = harmonized['HTSUS'].str.replace('.', '', regex=False).str[0:6]
harmonized = harmonized.fillna('')

harmonized['Instrumento'] = 'Exclusão Ordem 30/07'
harmonized['Tarifa Desc'] = 'Adicional de até 10%'
harmonized['Tarifa'] = 0.10
harmonized['Vigência'] = 'Vigente'

lista_aviacao = harmonized[['SH6','Articles of Civil Aircraft only']].query('`Articles of Civil Aircraft only`=="*"')
harmonized = harmonized.query('`Articles of Civil Aircraft only`!="*"')[['SH6','Instrumento','Tarifa', 'Tarifa Desc','Vigência']].drop_duplicates()

lista_aviacao = lista_aviacao.drop_duplicates()

aco = pd.read_excel('./dados/comex/NCMs de aço e aluminio afetadas Seçao 232 EUA (2).xlsx',dtype=str).rename(columns={'SH_6' : 'SH6'})[['SH6']].drop_duplicates()

aco['Instrumento'] = 'Seção 232'
aco['Tarifa Desc'] = "25% (atual) a 50%"
aco['Tarifa'] = 0.50
aco['Vigência'] = 'Em análise'


  


tarifas = pd.concat([harmonized[['SH6','Instrumento','Tarifa','Vigência','Tarifa Desc']],aco[['SH6','Instrumento','Tarifa','Vigência','Tarifa Desc']],tarifas_auto[['SH6','Instrumento','Tarifa','Vigência','Tarifa Desc']]])#,automoveis[['SH6','Instrumento','Tarifa']]])
tarifas = tarifas.drop_duplicates()
ncm_sh6 = ncm_sh6.merge(tarifas, on='SH6', how='left')
ncm_sh6 = ncm_sh6.merge(lista_aviacao, on='SH6', how='left')
ncm_sh6['Articles of Civil Aircraft only'] = ncm_sh6['Articles of Civil Aircraft only'].fillna('')
ncm_sh6['Instrumento'] = ncm_sh6['Instrumento'].fillna('Ordem 30/07')
ncm_sh6['Tarifa Desc'] = ncm_sh6['Tarifa Desc'].fillna('Tarifa de 10+40%')
ncm_sh6['Tarifa'] = ncm_sh6['Tarifa'].fillna(0.5)
ncm_sh6['Vigência'] = ncm_sh6['Vigência'].fillna('Vigente')


In [10]:
ncm_sh6.loc[ncm_sh6['SH6'].str.startswith('74'),'Instrumento'] = 'Seção 232'
ncm_sh6.loc[ncm_sh6['SH6'].str.startswith('74'),'Tarifa Desc'] = '25% (atual) a 50%'
ncm_sh6.loc[ncm_sh6['SH6'].str.startswith('74'),'Tarifa'] = 0.50
ncm_sh6.loc[ncm_sh6['SH6'].str.startswith('74'),'Vigência'] = 'Em análise'

In [11]:
ncm_sh6['Instrumento'].unique()

array(['Ordem 30/07', 'Seção 232', 'Exclusão Ordem 30/07'], dtype=object)

In [12]:
ncm_sh6.head(2)

,SH6,Instrumento,Tarifa,Vigência,Tarifa Desc,Articles of Civil Aircraft only
0,490110,Ordem 30/07,0.5,Vigente,Tarifa de 10+40%,
1,490191,Ordem 30/07,0.5,Vigente,Tarifa de 10+40%,


### Filtrar apenas tarifas acima dos demais países

In [15]:

ncm_sh6 = ncm_sh6.query('Instrumento == "Ordem 30/07"')

## Carregar dados de setores com participação americana na exportação e verificar o total de impostos 

In [13]:
exportacoes_pia = pd.read_excel('./Outputs/final/exportacao_pia_intensidade.xlsx',dtype={'cnae' : str,'CO_SH6' : str})

In [14]:
exportacoes_pia.shape

(3222, 10)

In [16]:
exportacoes_pia_tarifas = exportacoes_pia.merge(ncm_sh6.rename(columns={"SH6":"CO_SH6"}),on='CO_SH6',how='inner')

In [17]:
exportacoes_pia_tarifas.shape

(2911, 15)

In [18]:
exportacoes_pia_tarifas['Instrumento'].unique()

array(['Ordem 30/07'], dtype=object)

In [19]:
exportacoes_pia_tarifas.head(2)

,CO_SH6,CO_ISIC_CLASSE,R$ FOB Mundo,R$ FOB USA,participacao_usa,cnae,cnae_descricao,Receita líquida de vendas de produtos e serviços industriais,Total da receita líquida de vendas,Intensidade Tecnológica,Instrumento,Tarifa,Vigência,Tarifa Desc,Articles of Civil Aircraft only
0,521029,1312,421859.13,9055.20,0.021465,13,Fabricação de produtos têxteis,71933723000,74466387000,Média Baixa Intensidade Tecnológica,Ordem 30/07,0.5,Vigente,Tarifa de 10+40%,
1,521032,1312,724739.40,7702.31,0.010628,13,Fabricação de produtos têxteis,71933723000,74466387000,Média Baixa Intensidade Tecnológica,Ordem 30/07,0.5,Vigente,Tarifa de 10+40%,


In [20]:
participacao_producao_eua = exportacoes_pia_tarifas.groupby(['cnae','cnae_descricao','Total da receita líquida de vendas','Intensidade Tecnológica'])['R$ FOB USA'].sum().reset_index()
participacao_producao_eua['participacao_producao_eua'] = participacao_producao_eua['R$ FOB USA'] / participacao_producao_eua['Total da receita líquida de vendas']
participacao_producao_eua = participacao_producao_eua.drop(columns=['R$ FOB USA','Total da receita líquida de vendas'])


In [21]:
participacao_producao_eua.head(2)

,cnae,cnae_descricao,Intensidade Tecnológica,participacao_producao_eua
0,072,Extração de minerais metálicos não-ferrosos,Média Baixa Intensidade Tecnológica,0.001123
1,081,"Extração de pedra, areia e argila",Média Baixa Intensidade Tecnológica,0.003594


In [22]:
exportacoes_pia_tarifas = exportacoes_pia_tarifas[['CO_SH6','cnae','cnae_descricao','Intensidade Tecnológica','Instrumento','Tarifa','Vigência','Articles of Civil Aircraft only']].merge(participacao_producao_eua[['cnae','participacao_producao_eua']], on=['cnae'], how='left')

In [23]:
exportacoes_pia_tarifas.shape

(2911, 9)

In [24]:
exportacoes_usa_2024 = pd.read_excel('./dados/comex/Exportacao EUA 2024 NCM.xlsx')

In [25]:
exportacoes_usa_2024['SH6'] = exportacoes_usa_2024['Código NCM'].astype(str).str.zfill(8).str[0:6]

In [26]:
exportacoes_usa_2024.head(2)

,Código NCM,Descrição NCM,2024 - Valor US$ FOB,SH6
0,27090010,Óleos brutos de petróleo,5830983860,270900
1,72071200,Outros produtos semimanufaturados de ferro ou ...,2774470802,720712


In [27]:
exportacoes_usa_2024 = exportacoes_usa_2024.groupby('SH6')[['2024 - Valor US$ FOB']].sum().reset_index()

In [28]:
ncm_sh6.head(2)

,SH6,Instrumento,Tarifa,Vigência,Tarifa Desc,Articles of Civil Aircraft only
0,490110,Ordem 30/07,0.5,Vigente,Tarifa de 10+40%,
1,490191,Ordem 30/07,0.5,Vigente,Tarifa de 10+40%,


In [29]:
##Filtrar todos os NCM's que não iniciam com 0 ou 9 
lista=("0","9")
exportacoes_usa_2024.merge(ncm_sh6, on='SH6', how='outer',indicator=True).query('_merge == "both"')


,SH6,2024 - Valor US$ FOB,Instrumento,Tarifa,Vigência,Tarifa Desc,Articles of Civil Aircraft only,_merge
4,010121,1713599.0,Ordem 30/07,0.5,Vigente,Tarifa de 10+40%,,both
5,010129,3520510.0,Ordem 30/07,0.5,Vigente,Tarifa de 10+40%,,both
34,010619,344.0,Ordem 30/07,0.5,Vigente,Tarifa de 10+40%,,both
35,010620,24711.0,Ordem 30/07,0.5,Vigente,Tarifa de 10+40%,,both
39,010639,1864.0,Ordem 30/07,0.5,Vigente,Tarifa de 10+40%,,both
...,...,...,...,...,...,...,...,...
6464,970390,33687145.0,Ordem 30/07,0.5,Vigente,Tarifa de 10+40%,,both
6465,970400,50.0,Ordem 30/07,0.5,Vigente,Tarifa de 10+40%,,both
6467,970510,1962.0,Ordem 30/07,0.5,Vigente,Tarifa de 10+40%,,both
6473,970610,857301.0,Ordem 30/07,0.5,Vigente,Tarifa de 10+40%,,both


In [30]:
ncm = pd.read_csv('./dados/comex/NCM (2).csv',sep=';',dtype={'CO_SH6': str, 'CO_ISIC_CLASSE': str})
### Carregar a tabela de mapeamento de ISIC para CNAE do IBGE
mapeamento_isic = pd.read_excel('./dados/mapeamento/mapeamento_isic_cnae.xlsx', dtype=str).drop(columns=['Unnamed: 3','Observações'])
##### Fazer a primeira tentativa usando códigos ISIC a 3 dígitos
mapeamento_isic = mapeamento_isic[mapeamento_isic['código ISIC'].str.len() == 3]
mapeamento_isic.tail()
#### Eliminar códigos ISIC sem correspondência
mapeamento_isic = mapeamento_isic.dropna()
mapeamento_isic['CNAE2d'] = mapeamento_isic['código CNAE'].str[0:2]
mapeamento_isic['ISIC2d'] = mapeamento_isic['código ISIC'].str[0:2]
#### Verificar se há códigos ISIC a dois dígitos que correspondem a mais de uma cnae a dois dígitos
mapeamento_isic.groupby('ISIC2d')[['CNAE2d']].nunique().sort_values(by='CNAE2d', ascending=False).head(10)
#### Códigos a 2 dígitos ISIC que estão relacionados a mais de um código CNAE a 2 dígitos sempre estão vinculados a dígitos idênticos mais um outro. Nesses casos, se considerará apenas os dois dígitos idênticos, descartando o segundo
mapeamento_isic = mapeamento_isic.query('CNAE2d == ISIC2d')
### Remover exportações com valor 0 e mapear exportações com código ISIC a 4 dígitos

mapeamento_isic['cnae'] = mapeamento_isic['código CNAE'].str.replace('.','', regex=False)
#### Verificar as situações em que um ISIC a 3 dígitos é mapeado para mais de uma CNAE


In [41]:
pd.read_excel('./dados/mapeamento/mapeamento_isic_cnae.xlsx', dtype=str).drop(columns=['Unnamed: 3','Observações']).query('`código ISIC`.str.startswith("303")')


,código ISIC,denominação ISIC,código CNAE,denominação CNAE
487,303,"Fabricación de aeronaves, naves espaciales y m...",30.4,Fabricação de aeronaves
488,3030,"Fabricación de aeronaves, naves espaciales y m...",30.41-5,Fabricação de aeronaves
489,3030,"Fabricación de aeronaves, naves espaciales y m...",30.42-3,"Fabricação de turbinas, motores e outros compo..."


In [32]:
ncm_isic = pd.read_csv('./dados/comex/NCM_ISIC (1).csv',sep=';',dtype={'CO_ISIC_CLASSE': str, 'NO_ISIC_CLASSE': str}    )
ncm_isic.query('CO_ISIC_CLASSE == "899"', engine='python')

,CO_ISIC_CLASSE,NO_ISIC_CLASSE,NO_ISIC_CLASSE_ING,NO_ISIC_CLASSE_ESP,CO_ISIC_GRUPO,NO_ISIC_GRUPO,NO_ISIC_GRUPO_ING,NO_ISIC_GRUPO_ESP,CO_ISIC_DIVISAO,NO_ISIC_DIVISAO,NO_ISIC_DIVISAO_ING,NO_ISIC_DIVISAO_ESP,CO_ISIC_SECAO,NO_ISIC_SECAO,NO_ISIC_SECAO_ING,NO_ISIC_SECAO_ESP


In [33]:
ncm.query('CO_ISIC_CLASSE.str.startswith("899")').head(2)

,CO_NCM,CO_UNID,CO_SH6,CO_PPE,CO_PPI,CO_FAT_AGREG,CO_CUCI_ITEM,CO_CGCE_N3,CO_SIIT,CO_ISIC_CLASSE,CO_EXP_SUBSET,NO_NCM_POR,NO_NCM_ESP,NO_NCM_ING
44,55051000,10,550510,3075.0,3075.0,3.0,26721,230,4000.0,8999,1601.0,Desperdícios de fibras sintéticas (incluindo o...,Desperdicios de fibras sintéticas,Waste of synthetic fibres
45,55052000,10,550520,3075.0,3075.0,3.0,26722,230,4000.0,8999,1601.0,Desperdícios de fibras artificiais (incluindo ...,Desperdicios de fibras artificiales,Waste of articificial fibres


In [34]:
ncm['CO_ISIC_CLASSE3d'] = ncm['CO_ISIC_CLASSE'].str[0:3]
ncm[['CO_SH6','CO_ISIC_CLASSE3d']].drop_duplicates().groupby('CO_SH6')[['CO_ISIC_CLASSE3d']].nunique().reset_index().sort_values(by='CO_ISIC_CLASSE3d', ascending=False).head(10)

,CO_SH6,CO_ISIC_CLASSE3d
6616,999999,1
0,010110,1
1,010111,1
2,010119,1
3,010120,1
4,010121,1
5,010129,1
6,010130,1
7,010190,1
6600,970529,1


In [35]:
mapeamento_isic_ncm = mapeamento_isic.merge(ncm[['CO_ISIC_CLASSE3d','CO_SH6']].drop_duplicates(), left_on='código ISIC', right_on='CO_ISIC_CLASSE3d', how='inner')

In [36]:
mapeamento_isic_ncm.query('CNAE2d == ISIC2d').shape

(14482, 9)

In [37]:
mapeamento_isic_ncm.shape

(14482, 9)

In [42]:
mapeamento_isic_ncm.head(2)

,código ISIC,denominação ISIC,código CNAE,denominação CNAE,CNAE2d,ISIC2d,cnae,CO_ISIC_CLASSE3d,CO_SH6
0,011,Cultivo de plantas no perenes,01.1,Produção de lavouras temporárias,01,01,011,011,520100
1,011,Cultivo de plantas no perenes,01.1,Produção de lavouras temporárias,01,01,011,011,070956


In [43]:
lista_isic_unica_cnae = mapeamento_isic_ncm.groupby('código ISIC')[['cnae']].nunique().query('cnae == 1').reset_index()['código ISIC'].tolist()

In [44]:
mapeamento_isic3d = pd.read_excel('./dados/mapeamento/mapeamento_isic_cnae.xlsx', dtype=str).drop(columns=['Unnamed: 3','Observações'])
mapeamento_isic3d['código ISIC'] = mapeamento_isic3d['código ISIC'].str.replace('.','', regex=False)
mapeamento_isic3d['código CNAE'] = mapeamento_isic3d['código CNAE'].str.replace('.','', regex=False)

mapeamento_isic3d = mapeamento_isic3d[mapeamento_isic3d['código ISIC'].str.len() == 3]

In [45]:
mapeamento_isic3d.query('`código ISIC` in @lista_isic_unica_cnae', engine='python').groupby('código CNAE')[['código ISIC']].nunique().reset_index().sort_values(by='código ISIC', ascending=False)

,código CNAE,código ISIC
19,154,3
6,060,2
16,106,2
17,141,2
5,050,2
...,...,...
62,531,1
63,591,1
64,742,1
65,900,1


In [46]:
#Para valores não existentes em lista_isic_unica_cnae, considerar o cnae a 2 dígitos
mapeamento_isic_ncm['cnae'] = mapeamento_isic_ncm['cnae'].str[0:2].where(~mapeamento_isic_ncm['código ISIC'].isin(lista_isic_unica_cnae), mapeamento_isic_ncm['cnae'])

In [47]:
mapeamento_isic = pd.read_excel('./dados/mapeamento/mapeamento_isic_cnae.xlsx', dtype=str).drop(columns=['Unnamed: 3','Observações'])
mapeamento_isic['código ISIC'] = mapeamento_isic['código ISIC'].str.replace('.','', regex=False)
mapeamento_isic['código CNAE'] = mapeamento_isic['código CNAE'].str.replace('.','', regex=False)

mapeamento_isic = mapeamento_isic[mapeamento_isic['código ISIC'].str.len() == 2]

In [48]:
mapeamento_isic = mapeamento_isic[['código CNAE','denominação CNAE']]
mapeamento_isic['denominação CNAE'] = mapeamento_isic['denominação CNAE'].str.capitalize()


In [49]:
mapeamento_isic.head(2)

,código CNAE,denominação CNAE
1,01,"Agricultura, pecuária e serviços relacionados"
68,02,Produção florestal


In [50]:
mapeamento_isic = mapeamento_isic.drop_duplicates()

In [51]:
mapeamento_isic.head(2)['código CNAE'].values

array(['01', '02'], dtype=object)

In [53]:
mapeamento_isic_ncm.query('`código ISIC`.str.startswith("303")').head(2)

,código ISIC,denominação ISIC,código CNAE,denominação CNAE,CNAE2d,ISIC2d,cnae,CO_ISIC_CLASSE3d,CO_SH6
14103,303,"Fabricación de aeronaves, naves espaciales y m...",30.4,Fabricação de aeronaves,30,30,304,303,880260
14104,303,"Fabricación de aeronaves, naves espaciales y m...",30.4,Fabricação de aeronaves,30,30,304,303,840710


In [54]:
#Para cada código cnae que possuir comprimento 2 dígitos, substituir o campo 'denominação CNAE' pelo valor correspondente em mapeamento_isic, sem alterar o valor nos casos em que esse campo possua 3 dígitos
#Criar máscara para códigos CNAE de 2 dígitos
mask_2_digitos = mapeamento_isic_ncm['cnae'].str.len() == 2

# Verificar se o mapeamento existe antes de aplicar
if mask_2_digitos.any():
    # Fazer o mapeamento
    mapeamento_dict = mapeamento_isic.set_index('código CNAE')['denominação CNAE'].to_dict()
    mapeamento_isic_ncm.loc[mask_2_digitos, 'denominação CNAE'] = mapeamento_isic_ncm.loc[mask_2_digitos, 'cnae'].map(mapeamento_dict)
    
    # Verificar se ainda há valores nulos após o mapeamento
    nulos_restantes = mapeamento_isic_ncm.loc[mask_2_digitos, 'denominação CNAE'].isna().sum()
    if nulos_restantes > 0:
        print(f"Atenção: {nulos_restantes} códigos CNAE de 2 dígitos não foram mapeados")

In [55]:
mapeamento_isic_ncm = mapeamento_isic_ncm[['cnae', 'denominação CNAE','CO_SH6']].drop_duplicates()

In [56]:
mapeamento_isic_ncm.head(2)

,cnae,denominação CNAE,CO_SH6
0,01,"Agricultura, pecuária e serviços relacionados",520100
1,01,"Agricultura, pecuária e serviços relacionados",070956


In [57]:
exportacoes_usa_2024.merge(mapeamento_isic_ncm, left_on='SH6', right_on='CO_SH6', how='outer',indicator=True).value_counts('_merge')

_merge
both          3487
right_only    3019
left_only       39
Name: count, dtype: int64

In [58]:
exportacoes_usa_2024.merge(mapeamento_isic_ncm, left_on='SH6', right_on='CO_SH6', how='outer',indicator=True).query('_merge == "left_only"')['2024 - Valor US$ FOB'].sum()

np.float64(66715335.0)

In [59]:
exportacoes_usa_2024['2024 - Valor US$ FOB'].sum()

np.int64(40368569157)

In [60]:
100*66715335.0/40368569157

0.16526554295380919

In [61]:
exportacoes_usa_2024.head(2)

,SH6,2024 - Valor US$ FOB
0,010121,1713599
1,010129,3520510


In [62]:
exportacoes_usa_2024_isic = exportacoes_usa_2024.merge(mapeamento_isic_ncm, left_on='SH6', right_on='CO_SH6', how='inner')

In [63]:
exportacoes_usa_2024_isic.head(2)

,SH6,2024 - Valor US$ FOB,cnae,denominação CNAE,CO_SH6
0,010121,1713599,015,Pecuária,010121
1,010129,3520510,015,Pecuária,010129


In [64]:
exportacoes_usa_2024_isic.shape

(3487, 5)

In [65]:
ncm_sh6.merge(exportacoes_usa_2024_isic[['SH6']], on='SH6', how='inner').shape


(3165, 6)

In [66]:
ncm_sh6.head(2)

,SH6,Instrumento,Tarifa,Vigência,Tarifa Desc,Articles of Civil Aircraft only
0,490110,Ordem 30/07,0.5,Vigente,Tarifa de 10+40%,
1,490191,Ordem 30/07,0.5,Vigente,Tarifa de 10+40%,


In [67]:
exportacoes_usa_2024_isic = ncm_sh6.merge(exportacoes_usa_2024_isic, on='SH6', how='inner')

In [68]:
exportacoes_usa_2024_isic.tail()

,SH6,Instrumento,Tarifa,Vigência,Tarifa Desc,Articles of Civil Aircraft only,2024 - Valor US$ FOB,cnae,denominação CNAE,CO_SH6
3160,400122,Ordem 30/07,0.5,Vigente,Tarifa de 10+40%,,259693,01,"Agricultura, pecuária e serviços relacionados",400122
3161,400129,Ordem 30/07,0.5,Vigente,Tarifa de 10+40%,,10477,02,Produção florestal,400129
3162,392113,Ordem 30/07,0.5,Vigente,Tarifa de 10+40%,,33014,222,Fabricação de produtos de material plástico,392113
3163,481014,Ordem 30/07,0.5,Vigente,Tarifa de 10+40%,,107,17,"Fabricação de celulose, papel e produtos de papel",481014
3164,392112,Ordem 30/07,0.5,Vigente,Tarifa de 10+40%,,3742,222,Fabricação de produtos de material plástico,392112


In [69]:
##Considerar apenas valores acima da linha base
exportacoes_usa_2024_isic['Tarifa'] = 0.4

In [70]:
exportacoes_usa_2024_isic['Vigência'].drop_duplicates()

0    Vigente
Name: Vigência, dtype: object

In [77]:
exportacoes_usa_2024_isic.query('cnae == "265"')

,SH6,Instrumento,Tarifa,Vigência,Tarifa Desc,Articles of Civil Aircraft only,2024 - Valor US$ FOB,cnae,denominação CNAE,CO_SH6
57,901210,Ordem 30/07,0.4,Vigente,Tarifa de 10+40%,,18608,265,Fabricação de aparelhos e instrumentos de medi...,901210
58,901290,Ordem 30/07,0.4,Vigente,Tarifa de 10+40%,,12075,265,Fabricação de aparelhos e instrumentos de medi...,901290
378,902410,Ordem 30/07,0.4,Vigente,Tarifa de 10+40%,,98458,265,Fabricação de aparelhos e instrumentos de medi...,902410
379,910211,Ordem 30/07,0.4,Vigente,Tarifa de 10+40%,,45140,265,Fabricação de aparelhos e instrumentos de medi...,910211
380,910212,Ordem 30/07,0.4,Vigente,Tarifa de 10+40%,,13644,265,Fabricação de aparelhos e instrumentos de medi...,910212
...,...,...,...,...,...,...,...,...,...,...
2538,910700,Ordem 30/07,0.4,Vigente,Tarifa de 10+40%,,1164,265,Fabricação de aparelhos e instrumentos de medi...,910700
2546,903190,Ordem 30/07,0.4,Vigente,Tarifa de 10+40%,*,582108,265,Fabricação de aparelhos e instrumentos de medi...,903190
2547,903210,Ordem 30/07,0.4,Vigente,Tarifa de 10+40%,*,788622,265,Fabricação de aparelhos e instrumentos de medi...,903210
2548,903220,Ordem 30/07,0.4,Vigente,Tarifa de 10+40%,*,483106,265,Fabricação de aparelhos e instrumentos de medi...,903220


In [78]:
exportacoes_usa_2024_isic_agrupado = exportacoes_usa_2024_isic.groupby(['Instrumento','Tarifa Desc','Tarifa','Vigência','cnae','denominação CNAE','Articles of Civil Aircraft only'])[['2024 - Valor US$ FOB']].sum().reset_index()

In [79]:
exportacoes_usa_2024_isic_agrupado.to_excel('./Outputs/final/exportacoes_usa_2024_isic_agrupado.xlsx', index=False)

In [80]:
exportacoes_usa_2024_isic_agrupado.head(2)

,Instrumento,Tarifa Desc,Tarifa,Vigência,cnae,denominação CNAE,Articles of Civil Aircraft only,2024 - Valor US$ FOB
0,Ordem 30/07,Tarifa de 10+40%,0.4,Vigente,01,"Agricultura, pecuária e serviços relacionados",,2160136481
1,Ordem 30/07,Tarifa de 10+40%,0.4,Vigente,014,Produção de sementes e mudas certificadas,,2579602


In [113]:
exportacoes_usa_2024_isic_agrupado['Fob Tarifado'] = exportacoes_usa_2024_isic_agrupado['2024 - Valor US$ FOB'] * ( exportacoes_usa_2024_isic_agrupado['Tarifa'])

In [117]:
#Criar uma outra coluna que aplica a tarifa ao valor FOB apenas nos casos em que a tarifa de 10% se Articles of Civil Aircraft only seja igual "*"
exportacoes_usa_2024_isic_agrupado['Fob Tarifado Aviação'] = exportacoes_usa_2024_isic_agrupado['2024 - Valor US$ FOB'] * ( 0 * (exportacoes_usa_2024_isic_agrupado['Articles of Civil Aircraft only'] == '*').astype(int))


In [118]:
exportacoes_usa_2024_isic_agrupado = exportacoes_usa_2024_isic_agrupado.query('Instrumento != "Exclusão Ordem 30/07"')

In [119]:
exportacoes_usa_2024_isic_agrupado.head(2)

,Instrumento,Tarifa Desc,Tarifa,Vigência,cnae,denominação CNAE,Articles of Civil Aircraft only,2024 - Valor US$ FOB,Fob Tarifado,Fob Tarifado Aviação
0,Ordem 30/07,Tarifa de 10+40%,0.4,Vigente,01,"Agricultura, pecuária e serviços relacionados",,2160136481,864054592.4,0
1,Ordem 30/07,Tarifa de 10+40%,0.4,Vigente,014,Produção de sementes e mudas certificadas,,2579602,1031840.8,0


In [120]:
exportacoes_usa_2024_isic_agrupado_aviacao = exportacoes_usa_2024_isic_agrupado.query('`Articles of Civil Aircraft only` == "*"')
exportacoes_usa_2024_isic_agrupado_todos = exportacoes_usa_2024_isic_agrupado.copy()

In [121]:
exportacoes_usa_2024_isic_agrupado_todos = exportacoes_usa_2024_isic_agrupado_todos.groupby(['cnae','denominação CNAE'])[['2024 - Valor US$ FOB','Fob Tarifado']].sum().reset_index()
exportacoes_usa_2024_isic_agrupado_aviacao = exportacoes_usa_2024_isic_agrupado_aviacao.groupby(['cnae','denominação CNAE'])[['2024 - Valor US$ FOB','Fob Tarifado Aviação']].sum().reset_index()

In [122]:
exportacoes_usa_2024_isic_agrupado_todos.head(2)

,cnae,denominação CNAE,2024 - Valor US$ FOB,Fob Tarifado
0,01,"Agricultura, pecuária e serviços relacionados",2160136481,864054592.4
1,014,Produção de sementes e mudas certificadas,2579602,1031840.8


In [123]:
exportacoes_usa_2024_isic_agrupado_aviacao.query('cnae=="304"')

,cnae,denominação CNAE,2024 - Valor US$ FOB,Fob Tarifado Aviação
21,304,Fabricação de aeronaves,3006282021,0


In [124]:
agregacao_final = exportacoes_usa_2024_isic_agrupado_todos.merge(exportacoes_usa_2024_isic_agrupado_aviacao.rename(columns={'2024 - Valor US$ FOB' : 'FOB Potencial aviação'}),on=['cnae','denominação CNAE'],how='left')

In [125]:
agregacao_final.head(2)

,cnae,denominação CNAE,2024 - Valor US$ FOB,Fob Tarifado,FOB Potencial aviação,Fob Tarifado Aviação
0,01,"Agricultura, pecuária e serviços relacionados",2160136481,864054592.4,NaN,NaN
1,014,Produção de sementes e mudas certificadas,2579602,1031840.8,NaN,NaN


In [126]:
agregacao_final = agregacao_final.fillna(0.0)

In [127]:
agregacao_final['proporcao'] = agregacao_final['FOB Potencial aviação']/agregacao_final['2024 - Valor US$ FOB']

In [78]:
agregacao_final.columns

Index(['cnae', 'denominação CNAE', '2024 - Valor US$ FOB', 'Fob Tarifado',
       'FOB Potencial aviação', 'Fob Tarifado Aviação', 'proporcao'],
      dtype='object')

In [129]:
agregacao_final.head(2)

,cnae,denominação CNAE,2024 - Valor US$ FOB,Fob Tarifado,FOB Potencial aviação,Fob Tarifado Aviação,proporcao
0,01,"Agricultura, pecuária e serviços relacionados",2160136481,864054592.4,0.0,0.0,0.0
1,014,Produção de sementes e mudas certificadas,2579602,1031840.8,0.0,0.0,0.0


In [128]:
agregacao_final.query('cnae == "304"')

,cnae,denominação CNAE,2024 - Valor US$ FOB,Fob Tarifado,FOB Potencial aviação,Fob Tarifado Aviação,proporcao
58,304,Fabricação de aeronaves,3006282021,1.202513e+09,3.006282e+09,0.0,1.0


In [130]:
#Se proporcao for diferente de 0, Tarifa mínima deve ter o valor igual a (proporcao*FOB Potencial aviação)+((1-proporcao)*Fob Tarifado). Caso contrário, o valor é igual a Fob Tarifado 

import numpy as np

# Supondo que o seu DataFrame se chama df
agregacao_final['Tarifa mínima'] = np.where(
    agregacao_final['proporcao'] != 0,
    (agregacao_final['proporcao'] * agregacao_final['Fob Tarifado Aviação']) + ((1 - agregacao_final['proporcao']) * agregacao_final['Fob Tarifado']),
    agregacao_final['Fob Tarifado']
)



In [131]:
agregacao_final =  agregacao_final.rename(columns={'Fob Tarifado' : 'Tarifa máxima'})

### Usar dados de elasticidade de armington

In [132]:
armington = pd.read_excel('./dados/comex/cnaes vs inverso da elasticidade de armington.xlsx',dtype={'CNAE': str})

In [133]:
armington.head(2)

,CNAE,dificuldade de realocação
0,01,baixa
1,014,desconhecida


In [134]:
agregacao_final.shape

(73, 8)

In [135]:
agregacao_final.merge(armington,left_on='cnae', right_on='CNAE', how='left').head(2)

,cnae,denominação CNAE,2024 - Valor US$ FOB,Tarifa máxima,FOB Potencial aviação,Fob Tarifado Aviação,proporcao,Tarifa mínima,CNAE,dificuldade de realocação
0,01,"Agricultura, pecuária e serviços relacionados",2160136481,864054592.4,0.0,0.0,0.0,864054592.4,01,baixa
1,014,Produção de sementes e mudas certificadas,2579602,1031840.8,0.0,0.0,0.0,1031840.8,014,desconhecida


In [136]:
agregacao_final.head(2)

,cnae,denominação CNAE,2024 - Valor US$ FOB,Tarifa máxima,FOB Potencial aviação,Fob Tarifado Aviação,proporcao,Tarifa mínima
0,01,"Agricultura, pecuária e serviços relacionados",2160136481,864054592.4,0.0,0.0,0.0,864054592.4
1,014,Produção de sementes e mudas certificadas,2579602,1031840.8,0.0,0.0,0.0,1031840.8


In [137]:
agregacao_final_armington = agregacao_final.merge(armington,left_on='cnae', right_on='CNAE', how='left').rename(columns={'dificuldade de realocação': 'Dificuldade de realocação'})

In [138]:
agregacao_final_armington.drop(['FOB Potencial aviação','Fob Tarifado Aviação','proporcao','CNAE'],axis=1).to_excel('./Outputs/tarifa por setor armington.xlsx',index=False)

## Pedido Ad-hoc

In [ ]:
ncm_sh6_tarifas = ncm_sh6.merge(exportacoes_usa_2024[['SH6']], on='SH6', how='inner')
#Criar um arquivo excel e, para cada Instrumento de ncm_sh6_tarifas, criar uma aba apenas com os códigos CO_SH6
with pd.ExcelWriter('./Outputs/final/lista_SH6_por_tarifa.xlsx') as writer:
    for instrumento in ncm_sh6_tarifas['Instrumento'].unique():
        #Remover caracteres especiais do nome da aba
        instrumento_escape = instrumento.replace('/', '_').replace(' ', '_').replace('(', '').replace(')', '')
        ncm_sh6_tarifas.query('Instrumento == @instrumento')[['SH6']].to_excel(writer, sheet_name=instrumento_escape+'SH6', index=False)
        lista_sh6 = ncm_sh6_tarifas.query('Instrumento == @instrumento')['SH6'].unique()
        ncm.query('CO_SH6 in @lista_sh6')[['CO_NCM']].to_excel(writer, sheet_name=instrumento_escape+' NCM', index=False)

